In [5]:
import os
import sys

import pandas as pd
import numpy as np
from skimage.feature import graycomatrix, graycoprops

sys.path.insert(0, "\\".join(os.path.abspath(os.curdir).split("\\")[:-2:]))

from pydicom import dcmread

from src.utils.utils import *

In [6]:
path_json = "../../outputs/mamografia/extracted_metadata"
name_json = "extracted_metadata_CBIS-DDSM"

list_metadata = load_json(name_json, path_json)

images_original = []
images_resized = []
image_names = []
labels = []

for metadata in list_metadata:
    image_name = metadata['study_name']
    image_names.append(image_name)
    
    metadata_csv = metadata['metadata_csv']
    
    label = metadata_csv['pathology']
    labels.append(label)
    
    directory = Path(metadata_csv['original_image_path'])
    path_dicom_file = list(directory.rglob("*.dcm*"))[0]
    dicom_file = dcmread(path_dicom_file)
    
    # Carrega a matriz da imagem original e salva no array
    image = dicom_file.pixel_array
    # images_original.append(image)
    
    # Pega as dimensões da image
    height, width = image.shape
    ymin, ymax, xmin, xmax = height//3, height*2//3, width//3, width*2//3
    
    # Redimensiona a imagem original e salva no array
    image_255 = (image / image.max()) * 255
    image_255 = image_255.astype(np.uint8)
    crop = image_255[ymin:ymax, xmin:xmax]
    image_resized = cv2.resize(crop, (0,0), fx=0.5, fy=0.5)
    images_resized.append(image_resized)
    
images_resized[0].max()

141

In [7]:
properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']
glcm_features = [] # Cada uma das propriedades é trazida para a lista de features
angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]

glcm = graycomatrix(images_resized[0], distances=[1, 5], angles=angles, levels=256, symmetric=True, normed=True)
glcm_props = graycoprops(glcm, 'correlation')

for img, image_name in zip(images_resized, image_names):
    glcm = graycomatrix(img, distances=[1, 5], angles=angles, levels=256, symmetric=True, normed=True)
    feature = []
    glcm_props = [propery for name in properties for propery in graycoprops(glcm, name)]

    for glcm_props_distance in glcm_props:
        for item in glcm_props_distance:
            feature.append(item)

    glcm_features.append(feature)
 
columns = []
distances = ['1', '5']
angles = ['0', '45', '90', '135']

for name in properties:
    for distance in distances:
        for ang in angles:
            columns.append(name + "_ang_" + ang + "_dist_" + distance)

In [8]:
glcm_df = pd.DataFrame(glcm_features, columns = columns)
glcm_df['pathology'] = labels
""" glcm_df['pathology'] = glcm_df['pathology'].replace('BENIGN_WITHOUT_CALLBACK', 'BENIGN') """
glcm_df.to_csv("../../outputs/mamografia/matriz_features_glcm.csv", index=False)

glcm_df.head()

,dissimilarity_ang_0_dist_1,dissimilarity_ang_45_dist_1,dissimilarity_ang_90_dist_1,dissimilarity_ang_135_dist_1,dissimilarity_ang_0_dist_5,dissimilarity_ang_45_dist_5,dissimilarity_ang_90_dist_5,dissimilarity_ang_135_dist_5,correlation_ang_0_dist_1,correlation_ang_45_dist_1,...,ASM_ang_135_dist_5,energy_ang_0_dist_1,energy_ang_45_dist_1,energy_ang_90_dist_1,energy_ang_135_dist_1,energy_ang_0_dist_5,energy_ang_45_dist_5,energy_ang_90_dist_5,energy_ang_135_dist_5,pathology
0,0.867058,0.989288,0.869534,0.958089,1.558669,1.609807,1.330008,1.438225,0.998038,0.997318,...,0.374557,0.613519,0.612996,0.612982,0.613068,0.612237,0.611475,0.611724,0.612010,BENIGN
1,1.591570,1.756165,1.622403,1.851661,2.675529,2.538154,2.502785,2.851484,0.998133,0.997718,...,0.065602,0.259952,0.259348,0.260183,0.259283,0.255886,0.256496,0.258937,0.256129,BENIGN
2,0.880291,0.962439,0.879317,1.023709,1.631688,1.470731,1.372944,1.684637,0.997972,0.997525,...,0.356365,0.600135,0.599574,0.599586,0.599440,0.597867,0.597475,0.597672,0.596963,BENIGN_WITHOUT_CALLBACK
3,0.880291,0.962439,0.879317,1.023709,1.631688,1.470731,1.372944,1.684637,0.997972,0.997525,...,0.356365,0.600135,0.599574,0.599586,0.599440,0.597867,0.597475,0.597672,0.596963,BENIGN_WITHOUT_CALLBACK
4,1.496904,1.725108,1.533977,1.677406,2.473641,2.634147,2.381861,2.410005,0.998009,0.997348,...,0.090927,0.304731,0.304179,0.304973,0.304221,0.300906,0.301545,0.303951,0.301541,BENIGN_WITHOUT_CALLBACK
